# A Dual Approach to Holistic Regression
## March 4th, 2021

In [4]:
using Random, Distributions
using LinearAlgebra
using Gurobi, JuMP
using DataFrames
using CSV
using StatsBase
using Plots
using ProgressBars
using Optim

In [5]:
gurobi_env = Gurobi.Env()

function create_gurobi_model(; TimeLimit=-1, LogFile=nothing)
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env)));
    if TimeLimit >= 0
        println("Set Gurobi TimeLimit.")
        set_optimizer_attribute(model, "TimeLimit", TimeLimit)
    end
    if LogFile != nothing
        println("LogFile: $(LogFile).")
        set_optimizer_attribute(model, "LogFile", LogFile)
    else
        set_optimizer_attribute(model, "OutputFlag", 0)
    end
    set_optimizer_attribute(model, "NumericFocus", 3)
    return model
end;

Academic license - for non-commercial use only


____
## 0. Utils

In [6]:
function write_list(file_path, l)
    if length(l) == 0
        return
    end
    open(file_path, "a+") do io
        for e in l[1:end-1]
            try 
                e = round(e, digits=3)
                catch error end
            write(io, "$(e),")
        end
        e = l[end]
        try 
            e = round(e, digits=3)
        catch error end
        write(io, "$(e)\n")
    end
    return
end

function write_to_file(file_path, str)
    open(file_path, "a+") do io
        write(io, str)
    end
end

write_to_file (generic function with 1 method)

In [7]:
function get_support(s)
    supp = similar(s, Int)
    count_supp = 1
    
    supp_c = similar(s, Int)
    count_supp_c = 1
    
    @inbounds for i in eachindex(s)
        supp[count_supp] = i
        supp_c[count_supp_c] = i
        is_zero = s[i] < 0.5
        count_supp += !is_zero
        count_supp_c += is_zero
    end
    return resize!(supp, count_supp-1), resize!(supp_c, count_supp_c-1)
end

get_support([0, 0, 1, 1, 0])

([3, 4], [1, 2, 5])

____
## 1. Generate Synthetic Data  

In [9]:
function generate_synthetic_data(n, p, k, NR; seed=-1)
    """
        n = num. of samples
        p = num. of features
        k = num. of non zero coefficients
        NR = noise ratio ~ σ_noise = NR * σ_y_true
    """
    if seed >= 0
        Random.seed!(seed)
    end
    
    # Generate PD matrix
    A = randn(p, p)
    A = A'*A
    Σ = (A' + A)/2
    
    # Generate data X
    d = MvNormal(Σ)
    X = rand(d, n)'I
    
    # Split data
    index_train = 1:floor(Int, 0.5*n)
    index_val = floor(Int, 0.5*n)+1:floor(Int, 0.75*n)
    index_test = floor(Int, 0.75*n)+1:n
    
    X_train = X[index_train,:]
    X_val = X[index_val,:]
    X_test = X[index_test,:]
    
    # Center
    μ_train = [mean(X_train[:, j]) for j=1:p]
    for j=1:p
         X_train[:,j] = X_train[:,j] .- μ_train[j]
         X_val[:,j] = X_val[:,j] .- μ_train[j]
         X_test[:,j] = X_test[:,j] .- μ_train[j]
    end
    
    # Scale
    σ_train = [norm(X_train[:, j]) for j=1:p]
    for j=1:p
         X_train[:,j] = X_train[:,j]/σ_train[j]
         X_val[:,j] = X_val[:,j] ./ σ_train[j]
         X_test[:,j] = X_test[:,j] ./ σ_train[j]
    end
    
    # Generate β
    β = zeros(p)
    for j=1:k
        β[floor(Int, j*p/k)] = 1.0*rand([-1, 1])
    end
    
    # Noise
    ϵ = rand(Normal(0, std(X*β)*NR), n)
    
    # Target
    y_train = X_train*β + ϵ[index_train]
    y_val = X_val*β + ϵ[index_val]
    y_test = X_test*β + ϵ[index_test]
            
    return  (X_train, y_train), (X_val, y_val), (X_test, y_test), β
end

function get_t_α(n, p, α)
    return quantile(TDist(n-p), 1 - α/2)
end

function get_σ_X(X, y, γ)
    n, p = size(X)
    
    # Estimator σ
    M_inv = inv(I/γ + X'X)
    σ_tilde = sqrt((y'*(I - X*M_inv*X')*y)/(n-p))
    σ_X = σ_tilde * sqrt.(diag(M_inv))
    
    return σ_X
end

function get_R2(y_pred, y_true, y_train)
    SS_res = norm(y_true .- y_pred)
    SS_tot = norm(y_true .- mean(y_train))
    return 1 - (SS_res/SS_tot)^2
end

;

## 2. Compute inner problems and gradients

### a. Compute g_s

In [10]:
function g_s(D_s, b_s, σ_X_s; GD=true)
    
    # Get length of support of s
    l = length(b_s)
    if l==0
        return zeros(0), 0.0
    end
    
    # Initial solution
    λ_s0 = zeros(l) .+ 1.0

    # Objective and gradient
    function fg!(F, G, λ_s)
        
        μ_s = λ_s .+ b_s
        β_s = D_s*μ_s
        
        if G != nothing
            G .= β_s .- σ_X_s
        end
        
        if F != nothing
            return -λ_s'σ_X_s + 0.5*μ_s'β_s
        end
    end
    
    lower = zeros(l)
    upper = [Inf for _ in 1:l]

    res = Optim.optimize(Optim.only_fg!(fg!), lower, upper, λ_s0, 
        Fminbox(GD ? GradientDescent() : LBFGS()))

    return Optim.minimizer(res), - Optim.minimum(res)
    
end;

In [11]:
function g_s_gurobi(D_s, b_s, σ_X_s, model)
    
    # Get length of support of s
    l = length(b_s)
    if l==0
        return zeros(0), 0.0
    end

    λ_s = model[:λ][1:l]
    μ_s = λ_s .+ b_s
    β_s = D_s*μ_s
    
    @objective(model, Max, λ_s'σ_X_s - 0.5*μ_s'β_s)
    
    optimize!(model)
    
    value.(λ_s), objective_value(model)
end;

In [12]:
function g_gurobi(supp, Z, D, b, σ_X, model)

    # Create DZ once
    DZ = D*Z
    
    λ = model[:λ]
    μ = b + λ

    @objective(model, Max, λ'*Z*σ_X - 0.5μ'*DZ*μ)
    
    optimize!(model)
    
    value.(λ)[supp], objective_value(model)
end;

### b. Compute ∇g_s

In [13]:
function ∇g_s(supp, supp_c, b, M, λ_s, D_s, σ_X_s)
    
    β_s = D_s*(b[supp] .+ λ_s)
  
    grad = zeros(length(b))
    grad[supp] = λ_s .* σ_X_s - (β_s .^ 2)/(2γ)
    grad[supp_c] = - 0.5*γ*(b[supp_c] - M[supp_c, supp]*β_s).^2
    
    return grad
    
end

∇g_s (generic function with 1 method)

In [14]:
function ∇g(supp, D, b, λ_s, σ_X)
    
    λ = zeros(length(b))
    λ[supp] = λ_s
    
    grad = λ .* σ_X - ((D'*(b + λ)).^ 2)/(2γ)

    return grad
    
end

∇g (generic function with 1 method)

_____
## 3. Compare speed 

### /!\ t_α is already in σ_X /!\

In [15]:
# Parameters
n_train = 10000
n = 2*n_train
p = 100
k = 10
NR = 0.001
α = 0.05
t_α = get_t_α(n, p, α)
γ = 100.0

# Generate data
(X_p, y), _, _, β_true = generate_synthetic_data(n, p, k, NR, seed=1997);
σ_X_p = t_α * get_σ_X(X_p, y, γ); #t_α is already in σ_X

# Compute data in p dimensions
M_p = X_p'X_p
b_p = X_p'y

# Compute data in 2p dimensions
M = [M_p -M_p; -M_p  M_p]
b = [b_p; -b_p];
σ_X = [σ_X_p; σ_X_p] ;

In [16]:
# Create s
s_true = vcat(β_true .> 0, β_true .< 0) .* 1 
supp, supp_c = get_support(s_true)

# # Random s satisfying constraints
# Random.seed!(1997)
# allowed_index = randperm(length(β_true))[1:k]
# β_rand = [rand()*rand([-1,1])*(x in allowed_index) for x in 1:length(β_true)]
# s_rand = vcat(β_rand .> 0, β_rand .< 0) .* 1 
# supp, supp_c = get_support(s_rand)

# # Quick check
# s_true = s_rand

# Get projected variables
b_s = b[supp];
σ_X_s = σ_X[supp];
D_s = inv(I/γ + M[supp, supp]);

# Create model for g_s_gurobi
model_inner_g_s = create_gurobi_model();
@variable(model_inner_g_s, λ[1:k] >= 0)

# Create model for g_gurobi
model_inner_g = create_gurobi_model();
@variable(model_inner_g, λ[1:2p] >= 0);

Z = Diagonal(s_true);
D = inv(I/γ + Z*M);

### Compare g_s

In [17]:
λ_s_GD, g_s_GD = g_s(D_s, b_s, σ_X_s; GD=true)
λ_s_LBFGS, g_s_LBFGS = g_s(D_s, b_s, σ_X_s; GD=false)
λ_s_guro_s, g_s_guro_s = g_s_gurobi(D_s, b_s, σ_X_s, model_inner_g_s)
λ_s_guro, g_s_guro = g_gurobi(supp, Z, D, b, σ_X, model_inner_g)

# Compare objective values
println("GD: ",g_s_GD)
println("LBFGS: ",g_s_LBFGS)
println("Gurobi (g_s): ", g_s_guro_s)
println("Gurobi (g): ", g_s_guro)

# Compare λ
hcat(λ_s_GD, λ_s_LBFGS, λ_s_guro_s, λ_s_guro)

GD: -5.173336913786829
LBFGS: -5.173336913786829
Gurobi (g_s): -5.173336919616882
Gurobi (g): -5.17333691961688


10×4 Array{Float64,2}:
 2.39651e-21  2.39838e-21  7.07466e-10  7.07466e-10
 2.3689e-21   2.37075e-21  7.35898e-10  7.35898e-10
 2.29811e-21  2.29991e-21  8.5358e-10   8.5358e-10 
 2.41443e-21  2.41632e-21  8.57559e-10  8.57559e-10
 2.14227e-21  2.14395e-21  5.8911e-10   5.8911e-10 
 2.15945e-21  2.16114e-21  6.76515e-10  6.76515e-10
 2.13979e-21  2.14147e-21  5.74114e-10  5.74114e-10
 2.37638e-21  2.37824e-21  7.4913e-10   7.4913e-10 
 2.31651e-21  2.31832e-21  7.4499e-10   7.4499e-10 
 2.28343e-21  2.28522e-21  8.11463e-10  8.11463e-10

In [18]:
# Computational time for D_s compared to D (given s)

total_time_D_s = 0
total_time_D = 0
for _ in 1:1000
    
    total_time_D_s += @elapsed begin 
        supp, supp_c = get_support(s_true)
        b_s = b[supp];
        σ_X_s = σ_X[supp];
        D_s = inv(I/γ + M[supp, supp]);
    end
    
    total_time_D += @elapsed begin
        Z = Diagonal(s_true);
        D = inv(I/γ + Z*M);
    end
end

println("Compute D_s: ", total_time_D_s)
println("Compute D: ", total_time_D)

Compute D_s: 0.035531820000000006
Compute D: 31.328474288000002


In [19]:
# Computational time of g_s and g for given D_s or D

total_time_GD = 0
total_time_LBFGS = 0
total_time_Gurobi_g_s = 0
total_time_Gurobi_g = 0

for _ in 1:1000
    total_time_GD += @elapsed g_s(D_s, b_s, σ_X_s; GD=true)
    total_time_LBFGS += @elapsed g_s(D_s, b_s, σ_X_s; GD=false)
    total_time_Gurobi_g_s += @elapsed g_s_gurobi(D_s, b_s, σ_X_s, model_inner_g_s)
    total_time_Gurobi_g += @elapsed g_gurobi(supp, Z, D, b, σ_X, model_inner_g)
end

println("Optim.jl + GD: ",total_time_GD)
println("Optim.jl + LBFGS: ", total_time_LBFGS)
println("Gurobi g_s (model created outside): ", total_time_Gurobi_g_s)
println("Gurobi g (model created outside): ", total_time_Gurobi_g)

Optim.jl + GD: 0.4123018070000006
Optim.jl + LBFGS: 0.7265850960000007
Gurobi g_s (model created outside): 0.9365747890000007
Gurobi g (model created outside): 33.58230823099998


### Compare ∇g_s

In [20]:
# Compare the gradient for different optimal lambdas

∇g_s_guro = ∇g_s(supp, supp_c, b, M, λ_s_guro, D_s, σ_X_s)
∇g_s_GD = ∇g_s(supp, supp_c, b, M, λ_s_GD, D_s, σ_X_s)
∇g_guro = ∇g(supp, D, b, λ_s_guro, σ_X)

println("|| ∇g_s_guro - ∇g_s_GD || =  ", norm(∇g_s_guro - ∇g_s_GD))
println("|| ∇g_s_guro - ∇g_guro || =  ", norm(∇g_s_guro - ∇g_guro))
hcat(∇g_s_GD, ∇g_s_guro, ∇g_guro)[1:10, :]

|| ∇g_s_guro - ∇g_s_GD || =  9.154866938521507e-9
|| ∇g_s_guro - ∇g_guro || =  7.799126426020777e-15


10×3 Array{Float64,2}:
 -0.0666634   -0.0666634   -0.0666634 
 -0.0346163   -0.0346163   -0.0346163 
 -0.00129684  -0.00129684  -0.00129684
 -0.0126555   -0.0126555   -0.0126555 
 -0.13699     -0.13699     -0.13699   
 -0.0303344   -0.0303344   -0.0303344 
 -0.0175715   -0.0175715   -0.0175715 
 -0.0879472   -0.0879472   -0.0879472 
 -0.0267486   -0.0267486   -0.0267486 
 -0.00457767  -0.00457767  -0.00457767

In [21]:
# Computational time for D_s compared to D (given s)

total_time_∇g_s = 0
total_time_∇g = 0

for _ in 1:1000
    
    total_time_∇g_s += @elapsed begin 
        ∇g_s(supp, supp_c, b, M, λ_s_GD, D_s, σ_X_s)
    end
    
    total_time_∇g += @elapsed begin
        ∇g(supp, D, b, λ_s_GD, σ_X)
    end
end

println("Compute ∇g_s: ", total_time_∇g_s)
println("Compute ∇g: ", total_time_∇g)

Compute ∇g_s: 0.043460404000000015
Compute ∇g: 0.04625124200000008


_____
## 4. Compute Cutting plane algorithm

In [22]:
function compute_primal(X, y, k, γ, σ_X; TimeLimit=-1, LogFile=nothing)
    
    n, p = size(X)
    
    model = create_gurobi_model(;LogFile=LogFile, TimeLimit=TimeLimit)

    # TODO: change big-M values
    M1 = 1000
    M2 = 1000

    @variable(model, β[i=1:p])
    @variable(model, s[i=1:p], Bin)
    @variable(model, b[i=1:p], Bin)

    @constraint(model, sum(s) <= k)
    
    @time @constraint(model, [i=1:p], β[i] <= M1*s[i])
    @time @constraint(model, [i=1:p], β[i] >= -M1*s[i])

    @time @constraint(model, [i=1:p], β[i]/σ_X[i] + M2*b[i] >= s[i])
    @time @constraint(model, [i=1:p], -β[i]/σ_X[i] + M2*(1-b[i]) >= s[i])
    M = X'X
    Xty = (X'y)
    @time @objective(model, Min, 0.5*(y'y - 2*Xty'*β + β'*M*β) + (0.5/γ)* sum(β[j]^2 for j=1:p))
        
    JuMP.optimize!(model)
    
    return objective_value(model), value.(β)
end

compute_primal (generic function with 1 method)

In [23]:
function compute_warm_start_primal(X, y, k, γ, σ_X, time_limit; LogFile=nothing)
    
    n, p = size(X)
    
    model = create_gurobi_model(;TimeLimit=time_limit, LogFile=LogFile)

    # TODO: change big-M values
    M1 = 1000
    M2 = 1000

    @variable(model, β[i=1:p])
    @variable(model, s[i=1:p], Bin)
    @variable(model, b[i=1:p], Bin)

    @constraint(model, sum(s) <= k)
    
    @constraint(model, [i=1:p], β[i] <= M1*s[i])
    @constraint(model, [i=1:p], β[i] >= -M1*s[i])

    @constraint(model, [i=1:p], β[i]/σ_X[i] + M2*b[i] >= s[i])
    @constraint(model, [i=1:p], -β[i]/σ_X[i] + M2*(1-b[i]) >= s[i])

    @objective(model, Min, 0.5*sum((y[i] - X[i,:]'β)^2 for i=1:n) + (0.5/γ)* sum(β[j]^2 for j=1:p))
    JuMP.optimize!(model)
    
    s_val = Int.(value.(s))
    b_val = Int.(value.(b))
    
    return vcat(s_val .* (b_val .== 0), s_val .* (b_val .== 1))
end

compute_warm_start_primal (generic function with 1 method)

In [24]:
function compute_dual(X_p, y, k, γ, σ_X_p; LogFile=nothing, WarmStart=nothing, TimeLimit=-1)
    """
    WarmStart ∈ {  nothing, :RidgeStart, :PrimalStart }
    """
    
    # Get dimensions
    n, p = size(X_p)
    
    # Constant
    C = 0.5*y'y #TODO: add it in the objecive in the end
 
    # Compute data in p dimensions
    M_p = X_p'X_p
    b_p = X_p'y

    # Compute data in 2p dimensions
    M = [M_p -M_p; -M_p  M_p]
    b = [b_p; -b_p];
    σ_X = [σ_X_p; σ_X_p] ;
    
    # Outer problem
    miop = create_gurobi_model(;LogFile=LogFile, TimeLimit=TimeLimit)
    @variable(miop, s[1:2p], Bin)
    @variable(miop, t >= -C)
    @constraint(miop, sum(s) <= k)
    @constraint(miop, [i=1:p], s[i]+s[p+i]<=1)
    
    # Initial solution
    s0 = zeros(2p) #TODO: change this
    
    # a. First k values
    s0[1:k] .= 1
    
    # b. Ridge regression
    if (WarmStart == :RidgeStart)
        println("Ridge Warm Start.")
        β_ridge = inv(I/γ + M_p)*b_p
        s0[findall(x -> x>0, β_ridge)] .= 1.0
        s0[findall(x -> x<0, β_ridge) .+ p] .= 1.0
    end
    
    # c. Primal solution + Time limit
    if (WarmStart == :PrimalStart)
        println("Primal Warm Start.")
        s0 = compute_warm_start_primal(X_p, y, k, γ, σ_X, 20; LogFile=LogFile)
    end

    # Initial cut
    supp, supp_c = get_support(s0)
    
    D_s, b_s, σ_X_s = inv(I/γ + M[supp, supp]), b[supp], σ_X[supp]
    
    λ_s0, g_s0 = g_s(D_s, b_s, σ_X_s; GD=true)
    ∇g_s0 = ∇g_s(supp, supp_c, b, M, λ_s0, D_s, σ_X_s)
    
    @constraint(miop, t >= g_s0 + dot(∇g_s0, s - s0))
    @objective(miop, Min, t + C)
    
    # Cutting planes    
    function outer_approximation(cb_data)
        
        s_val = [callback_value(cb_data, s[i]) for i=1:2p]
        
        supp, supp_c = get_support(s_val)

        D_s, b_s, σ_X_s = inv(I/γ + M[supp, supp]), b[supp], σ_X[supp]

        λ_s_val, g_s_val = g_s(D_s, b_s, σ_X_s; GD=true)
        ∇g_s_val = ∇g_s(supp, supp_c, b, M, λ_s_val, D_s, σ_X_s)
        
        con = @build_constraint(t >= g_s_val + dot(∇g_s_val, s - s_val))
        MOI.submit(miop, MOI.LazyConstraint(cb_data), con)
        
    end
    
    MOI.set(miop, MOI.LazyConstraintCallback(), outer_approximation)
    JuMP.optimize!(miop)
    
    
    s_opt = JuMP.value.(miop[:s])
    supp, supp_c = get_support(s_opt)
    
    D_s, b_s, σ_X_s = inv(I/γ + M[supp, supp]), b[supp], σ_X[supp]
    
    λ_s_opt, g_s_opt = g_s(D_s, b_s, σ_X_s; GD=true)
    
    β = zeros(2p)
    β[supp] = D_s*(λ_s_opt + b_s)
    

    return objective_value(miop), β[1:p] - β[p+1:end]
end

compute_dual (generic function with 1 method)

In [25]:
k_algo = 15

15

In [26]:
@time primal_obj, β_primal = compute_primal(X_p, y, k_algo, γ, σ_X_p; TimeLimit = 50, LogFile="nouveau_debug.txt")

Set Gurobi TimeLimit.
LogFile: nouveau_debug.txt.
  0.076562 seconds (216.05 k allocations: 11.478 MiB)
  0.207198 seconds (541.91 k allocations: 29.232 MiB, 15.93% gc time)
  0.065245 seconds (66.75 k allocations: 3.698 MiB)
  0.047677 seconds (98.60 k allocations: 5.726 MiB)
  0.201206 seconds (178.56 k allocations: 10.893 MiB, 6.53% gc time)
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 401 rows, 300 columns and 1100 nonzeros
Model fingerprint: 0x5860bb37
Model has 5050 quadratic objective terms
Variable types: 100 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [4e-03, 1e+00]
  QObjective range [2e-05, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+03]
Found heuristic solution: objective 10.3239397
Presolve time: 0.00s
Presolved: 401 rows, 300 columns, 1100 nonzeros
Presolved model has 5050 quadratic objective terms
Variable types: 100 continuous, 200 integer (

(5.150601260206673, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.956815  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.963714])

In [27]:
@time dual_obj, β_dual = compute_dual(X_p, y, k_algo, γ, σ_X_p; TimeLimit = 50, LogFile="nouveau_debug.txt")

Set Gurobi TimeLimit.
LogFile: nouveau_debug.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 102 rows, 201 columns and 601 nonzeros
Model fingerprint: 0x6feb2d70
Variable types: 1 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [2e-04, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [4e-01, 2e+01]
Presolve time: 0.00s
Presolved: 102 rows, 201 columns, 601 nonzeros
Variable types: 1 continuous, 200 integer (200 binary)

Root relaxation: objective 0.000000e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2          -    0.00000      -     -    0s
     0     2    0.00000    0    2          -    0.00000      -     -    0s
*  383   435              89      11.7218287    0.00000   100%   1.7    1s
*  580   518      

(7.3466563840949775, [-0.390899, 0.0, 0.0, 0.0, 0.364148, -0.435916, 0.0, 0.0, 0.0, -0.87792  …  0.0, 0.0, -0.207251, 0.0, 0.0, 0.182076, 0.0, 0.0, 0.0, 0.0])

In [ ]:
hcat(β_true, β_primal, β_dual)

_____
## 5. Experiences

In [99]:
# Parameters
n_train = 10000
n = 2*n_train
NR = 0.001
α = 0.05;
t_max = 30*60 # 30 min max per solving

1800

In [ ]:
csv_path = "results/2021_03_06_results.csv"
logfile = "results/2021_03_06_logs.txt"

write_list(csv_path, ["Algo", "Seed", "n", "p", "k_true", "k", "γ", 
                      "NR", "α", "R2", "OR2", "t_algo", "t_data", "t_variance"])

for seed ∈ [1997, 1998]
    for p ∈ [100, 150, 200, 300]

        # True sparsity
        k_true = div(p, 10)

        # Generate data
        t_data = @elapsed (X_train, y_train), _, (X_test, y_test), β_true = generate_synthetic_data(n, p, k_true, NR, seed=seed);

        # Significance
        t_variance = @elapsed t_α = get_t_α(n_train, p, α)
        
        # Robustness
        for γ ∈ [1.0, 10.0, 100.0]
            
            # Variance
            t_variance += @elapsed σ_X = t_α * get_σ_X(X_train, y_train, γ)
        
            # Estimated sparsity
            Random.seed!(seed)
            k = rand(5:15)
            
            # Dual
            t_dual = @elapsed obj_dual, β_dual = compute_dual(X_train, y_train, k, γ, σ_X; LogFile=logfile, TimeLimit=t_max);
            R2_dual = get_R2(X_train*β_dual, y_train, y_train)
            OR2_dual = get_R2(X_test*β_dual, y_test, y_train)
            write_list(csv_path, ["dual", seed, n_train, p, k_true, k, γ, 
                                  NR, α, R2_dual, OR2_dual, t_dual, t_data, t_variance])
            
            # Primal
            t_primal = @elapsed obj_primal, β_primal = compute_primal(X_train, y_train, k, γ, σ_X; LogFile=logfile, TimeLimit=t_max);
            R2_primal = get_R2(X_train*β_primal, y_train, y_train)
            OR2_primal = get_R2(X_test*β_primal, y_test, y_train)
            write_list(csv_path, ["primal", seed, n_train, p, k_true, k, γ, 
                                  NR, α, R2_primal, OR2_primal, t_primal, t_data, t_variance])
            
        end
    end
end

Set Gurobi TimeLimit.
LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 102 rows, 201 columns and 601 nonzeros
Model fingerprint: 0x33892736
Variable types: 1 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [6e-02, 2e+01]
Presolve time: 0.00s
Presolved: 102 rows, 201 columns, 601 nonzeros
Variable types: 1 continuous, 200 integer (200 binary)

Root relaxation: objective 3.344507e+00, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.94241    0    2          -    6.94241      -     -    0s
H    0     0                       7.4952511    6.94241  7.38%     -    0s
     0     2    7.41094    0    4    7.49525    7.41094  1.12%     -    0s
* 2504   

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 102 rows, 201 columns and 601 nonzeros
Model fingerprint: 0x4d7ae719
Variable types: 1 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [8e-03, 2e+01]
Presolve time: 0.00s
Presolved: 102 rows, 201 columns, 601 nonzeros
Variable types: 1 continuous, 200 integer (200 binary)

Root relaxation: objective 3.327392e+00, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.36791    0    2          -    4.36791      -     -    0s
H    0     0                       5.5728688    4.36791  21.6%     -    0s
H    0     0                       5.5597986    4.36791  21.4%     -    0s
H    0     0                   

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 102 rows, 201 columns and 601 nonzeros
Model fingerprint: 0x006d11e9
Variable types: 1 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [8e-02, 2e+01]
Presolve time: 0.00s
Presolved: 102 rows, 201 columns, 601 nonzeros
Variable types: 1 continuous, 200 integer (200 binary)

Root relaxation: objective 2.834492e+00, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.29443    0    2          -    4.29443      -     -    0s
H    0     0                       5.2557431    4.29443  18.3%     -    0s
H    0     0                       5.1789770    4.29443  17.1%     -    0s
     0     2    5.12123    0   

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 152 rows, 301 columns and 901 nonzeros
Model fingerprint: 0xe620552e
Variable types: 1 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [4e-07, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+01]
  RHS range        [2e-01, 2e+01]
Presolve time: 0.00s
Presolved: 152 rows, 301 columns, 901 nonzeros
Variable types: 1 continuous, 300 integer (300 binary)

Root relaxation: objective 1.076762e+01, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.01092    0    2          -   11.01092      -     -    0s
H    0     0                      18.3010594   11.01092  39.8%     -    0s
H    0     0                      16.6563274   11.01092  33.9%     -    0s
H    0     0                   

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 152 rows, 301 columns and 901 nonzeros
Model fingerprint: 0x3b4a93f2
Variable types: 1 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [5e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+01]
  RHS range        [1e-02, 2e+01]
Presolve time: 0.00s
Presolved: 152 rows, 301 columns, 901 nonzeros
Variable types: 1 continuous, 300 integer (300 binary)

Root relaxation: objective 1.101014e+01, 28 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.07958    0    2          -   11.07958      -     -    0s
     0     2   11.07958    0    2          -   11.07958      -     -    0s
*    5     0               1      13.1580512   13.15805  0.00%   8.4    0s

Cutting planes:
  Lazy constra

...

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 152 rows, 301 columns and 901 nonzeros
Model fingerprint: 0x4990c03a
Variable types: 1 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+01]
  RHS range        [2e-02, 6e+00]
Presolve time: 0.00s
Presolved: 152 rows, 301 columns, 901 nonzeros
Variable types: 1 continuous, 300 integer (300 binary)
Found heuristic solution: objective 17.3327147

Root relaxation: objective 1.358838e+01, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.55626    0    2   17.33271   14.55626  16.0%     -    0s
H    0     0                      14.5562585   14.55626  0.00%     -    0s

Cutting planes:
  Lazy constraints: 3

Explored 0 nodes (2

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 202 rows, 401 columns and 1201 nonzeros
Model fingerprint: 0x9cc27916
Variable types: 1 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [4e-06, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [6e-02, 6e+00]
Presolve time: 0.00s
Presolved: 202 rows, 401 columns, 1201 nonzeros
Variable types: 1 continuous, 400 integer (400 binary)

Root relaxation: objective 2.774321e+01, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.36920    0    2          -   28.36920      -     -    0s
H    0     0                      34.5516323   28.36920  17.9%     -    0s
     0     2   31.29486    0   12   34.55163   31.29486  9.43%     -    0s
*   57    54              20 

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 202 rows, 401 columns and 1201 nonzeros
Model fingerprint: 0x6dca9348
Variable types: 1 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [4e-07, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [4e-04, 6e+00]
Presolve time: 0.00s
Presolved: 202 rows, 401 columns, 1201 nonzeros
Variable types: 1 continuous, 400 integer (400 binary)

Root relaxation: objective 2.771604e+01, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.90790    0    2          -   27.90790      -     -    0s
     0     2   27.90790    0    2          -   27.90790      -     -    0s
H    5     4                      29.6207446   29.62074  0.00%   4.0    0s

Cutting planes:
  Lazy const

LogFile: results/2021_03_06_logs.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 202 rows, 401 columns and 1201 nonzeros
Model fingerprint: 0x48f63206
Variable types: 1 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [7e-06, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [4e-02, 6e+00]
Presolve time: 0.00s
Presolved: 202 rows, 401 columns, 1201 nonzeros
Variable types: 1 continuous, 400 integer (400 binary)

Root relaxation: objective 2.767882e+01, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.91861    0    4          -   27.91861      -     -    0s
H    0     0                      31.5372631   27.91861  11.5%     -    0s
H    0     0                      30.5561499   27.91861  8.63%     -    0s
H    0     0                 

Excessive output truncated after 532352 bytes.